# Demo de 'Transfer Learning con TF-Hub' en el cual se logra un modelo que puede aprender a reconocer los Tipos de Flores Iris usando como base un modelo ya entrenado para reconocer otros tipos de objetos
Basado en https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub

0) Instalar el paquete:

In [ ]:
# Luego de instalar el paquete tal vez necesario reiniciar el entorno (ver mensajes que genera)
try:
  # %tensorflow_version only exists in Colab.
  !pip install -q tf-nightly
except Exception:
  pass

!pip install -q -U tf-hub-nightly
!pip install -q tfds-nightly

1) Importar librerías:

In [ ]:
# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pylab as plt
from tensorflow.keras import layers

import numpy as np
import PIL.Image as Image
import csv

print("\nLibrerías importadas")

2) Cargar el modelo a utilizar para procesar:

In [ ]:
# selección del modelo a usar de base
#modeloUsar = 'mobilenet_v2'
modeloUsar = 'inception_v3'

print("\nModelo ", modeloUsar, "seleccionado")

In [ ]:
# define configuración del modelo
if modeloUsar == 'inception_v3':

    # URLs donde se encuentra la info del modelo Inception v3
    classifier_url ="https://tfhub.dev/google/tf2-preview/inception_v3/classification/4" 
    feature_extractor_url = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"

    # define el tamaño de imágenes soportadas del modelo Inception v3
    IMAGE_SHAPE = (299, 299)
else:

    # URLs donde se encuentra la info del modelo mobileNet
    classifier_url ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4" 
    feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

    # define el tamaño de imágenes soportadas del modelo mobileNet
    IMAGE_SHAPE = (224, 224)

# carga el módulo a usar
classifier = tf.keras.Sequential([ hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3,)) ])

# baja la lista de clases originales que maneja el modelo (inception o mobilenet)
model_labels_path = tf.keras.utils.get_file('ImageNetLabels.txt',
                                            'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
model_imagenet_labels = np.array(open(model_labels_path).read().splitlines())

print("\nModelo ", modeloUsar, "cargado")

3) Montar el Drive:

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# directorio local en Google Drive
path = 'gdrive/My Drive/IA/demo IRIS' 
model_export_path = path + '/Model_TFHub'
imagPath = path + '/IRIS/train'

4) Cargar imágenes para re-entrenar:

In [ ]:
# carga y aplica DataAugmentation 
# (los nombres de las clases son los nombres de los directorios)
image_generator =  tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                                   shear_range = 0.2,
                                                                   zoom_range = 0.2,
                                                                   horizontal_flip = False)

print("Imágenes: ")
image_data = image_generator.flow_from_directory(str(imagPath), 
                                                 target_size=IMAGE_SHAPE,
                                                 class_mode="categorical",
                                                 shuffle=True)
for image_batch, label_batch in image_data:
  print("Image batch shape: ", image_batch.shape)
  print("Label batch shape: ", label_batch.shape)
  break

  # define los nombres de las clases
dic_class_names = sorted(image_data.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in dic_class_names])
print("\nClases: ", class_names)

5) Probar resultados del modelo antes de entrenar ejecutando el modelo sobre las imágenes cargadas:

In [ ]:
# ejecuta el modelo
result_batch = classifier.predict(image_batch)

# determina resultados
predicted_class_names = model_imagenet_labels[np.argmax(result_batch, axis=-1)]

# muestra resultados
plt.figure(figsize=(10,19))
plt.subplots_adjust(hspace=0.4)
col = 3
rows=len(image_batch)//col

_ = plt.suptitle("Predicciones del Modelo Original")
for n in range(col*rows):
  plt.subplot(rows,col,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_class_names[n])
  plt.axis('off')


6) Re-entrenar el modelo:

In [ ]:
# Obtiene el modeo base del modelo para usar en el re-entrenamiento, congelandolo
# y agregandole una nueva capa para aprender las nuevas clases

# Create the feature extractor
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=IMAGE_SHAPE+(3,))

# It returns a 1280-length vector for each image:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

# Freeze the variables in the feature extractor layer, so that the training only modifies the new classifier layer.
feature_extractor_layer.trainable = False

# Attach a classification head
model = tf.keras.Sequential([ feature_extractor_layer, 
                             layers.Dense(image_data.num_classes, activation='softmax') ])

# Use compile to configure the training process
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss='categorical_crossentropy',
  metrics=['acc'] )

# nuevo modelo a re-entrenar
print("\n")
model.summary()

# info de imágenes a aprender
print("\n")
predictions = model(image_batch)
predictions.shape

In [ ]:
# ejecuta el re-entrenamiento del modelo

# functions to visualize the training progress
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []

  def on_train_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    self.model.reset_metrics()

# ejecuta el entrenamiento
steps_per_epoch = np.ceil(image_data.samples/image_data.batch_size)

batch_stats_callback = CollectBatchStats()

history = model.fit_generator(image_data, epochs=35,
                              steps_per_epoch=steps_per_epoch,
                              callbacks = [batch_stats_callback])

print("\nModelo re-entrenado")

In [ ]:
# muestra gráficos con resultados del re-entrenamiento
plt.figure()
plt.ylabel("Perdida")
plt.xlabel("Pasos de Entrenamiento")
plt.ylim([0,2])
plt.plot(batch_stats_callback.batch_losses)

plt.figure()
plt.ylabel("Exactitud")
plt.xlabel("Pasos de Entrenamiento")
plt.ylim([0,1])
plt.plot(batch_stats_callback.batch_acc)

7) Probar nuevo modelo re-entrenado:

In [ ]:
# ejecuta el modelo re-entrenado
predicted_batch = model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
label_id = np.argmax(label_batch, axis=-1)

# muestra resultados
plt.figure(figsize=(10,19))
plt.subplots_adjust(hspace=0.4)
col = 3
rows = len(image_batch)//col

for n in range(col*rows):
  plt.subplot(rows,col,n+1)
  plt.imshow(image_batch[n])
  if  predicted_id[n] == label_id[n]:
    color = "green"  
    res = class_names[predicted_id[n]]
  else:
    color = "red"
    res = class_names[predicted_id[n]] + ' [real ' + class_names[label_id[n]] + ']' 

  plt.title(res, color=color)
  plt.axis('off')
_ = plt.suptitle('Predicciones del Modelo ' + modeloUsar + ' Re-Entrenado (verde ok, rojo error)')


8) Grabar el modelo re-entrenado:

In [ ]:
# exporta modelo reentrenado
model.save(model_export_path, save_format='tf')
print("\nModelo grabado en ", model_export_path)

# exporta definición de tamaño de imágenes
with open( model_export_path + '/imagshape.csv', mode='w') as csvfile:
    wr = csv.writer(csvfile)
    wr.writerow(IMAGE_SHAPE)

# exporta definición de las clases
with open( model_export_path + '/clases.csv', mode='w') as csvfile:
    wr = csv.writer(csvfile)
    wr.writerow(class_names)
print('Definición de las clases: ', class_names, ' grabada')